
<center>
<img src="https://avatars.githubusercontent.com/u/86923959?s=400&u=3097a31c5eb9e29e67fe5fd285373b52ecdae5e3&v=4" width="15%" />
</center>
 


### Here we will start by making the connection to a SQLite database named 'jazz_projet.db' and load a schema from a SQL file named 'DATA.sql'.

        Steps:
            1. Connect to the SQLite database 'jazz_projet.db'.
            2. Open and read the content of 'DATA.sql'.
            3. Split the SQL file content into individual SQL commands.
            4. Execute each SQL command using the database cursor.
            5. If a command fails due to an OperationalError, it is ignored            and a message is printed.

In [1]:
import sqlite3

db_jazz = sqlite3.connect('jazz_projet.db')
cursor = db_jazz.cursor()

# Loading the schema into the database
fd = open('DATA.sql', 'r')
sqlFile = fd.read()
fd.close()

sqlCommands = sqlFile.split(';')

for command in sqlCommands:
    try:
        cursor.execute(command)
    except sqlite3.OperationalError:
        print('Command skipped: ' + command)

In [2]:
tables = ['Salle', 'Concert', 'Client', 'Musicien', 'Morceau', 'Organisateur', 'Reservation', 'Participation', 'Programme', 'Auteur', 'Seance']
for table in tables:
    cursor.execute(f"SELECT * FROM {table}")
    rows = cursor.fetchall()

    print("\n--- TABLE", table, "\n")

    for desc in cursor.description:
        print(desc[0].rjust(22, ' '), end=' ')

    print("")



--- TABLE Salle 

              ID_salle              Nom_salle          Nombre_Places 

--- TABLE Concert 

            ID_concert            Nom_concert            ID_musicien        ID_organisateur 

--- TABLE Client 

             ID_client             Nom_client             Tel_client 

--- TABLE Musicien 

           ID_musicien           Nom_musicien           Tel_musicien 

--- TABLE Morceau 

            ID_morceau            Nom_morceau 

--- TABLE Organisateur 

       ID_organisateur       Nom_organisateur       Tel_organisateur 

--- TABLE Reservation 

            ID_concert                   Date              ID_client              Num_place                   Prix 

--- TABLE Participation 

           ID_musicien             ID_concert 

--- TABLE Programme 

            ID_morceau             ID_concert           ID_arrangeur 

--- TABLE Auteur 

           ID_musicien             ID_morceau 

--- TABLE Seance 

            ID_concert                   Date           

In [3]:
def printTable(table):
    cursor.execute(f"SELECT * FROM {table}")
    rows = cursor.fetchall()
    print("\n--- TABLE", table, "\n")
    for desc in cursor.description:
        print(desc[0].rjust(22, ' '), end=' ')
    print("")
    for row in rows[:5]:
        for value in row:
            print(str(value).rjust(22, ' '), end=' ')
        print("")


<center>
<img src="images/Database Model.png" width="80%" />
</center>

### Now we insert data from text files into the corresponding tables of a database.

#### Variables

- **fichiers** (list): A list of filenames (without extensions) to process.
- **chemin** (str): The path to the directory containing the data files.

#### Features

- Opens the file `xJazz_clients_all.txt` from the specified path.
- Reads each line of the file, removes leading and trailing whitespace, and splits the line by semicolons.
- Executes an SQL `INSERT` statement to insert the data into the `Client` table.
- Commits the transaction to the database after each insertion.

In [4]:
fichiers = ['xJazz_clients_all', 'xJazz_morceux_et_arr_all', 'xJazz_musiciens_all', 'xJazz_organisateurs_all', 'xJazz_salles_all']

path = 'Jazz_data/'

# Inserts data from 'xJazz_clients_all.txt' into the 'Client' table
with open(path + 'xJazz_clients_all.txt', 'r') as f:
    for line in f:
        line = line.strip()
        line = line.split(';')
        cursor.execute("INSERT INTO Client VALUES (?, ?, ?)", (line[0], line[1], line[2]))
        db_jazz.commit()


### Insert `ID_Morceau` and `nom_morceau` into the `Morceau` table.

In [ ]:
with open(path + 'xJazz_morceux_et_arr_all.txt', 'r') as f:
    for line in f:
        line = line.strip()
        line = line.split(';')
        nom_morceau = line[1].split('\t')[0].replace('"', '')
        cursor.execute("INSERT INTO Morceau VALUES (?, ?)", (line[0].strip(), nom_morceau.strip()))
        db_jazz.commit()


### Insert artists into the `Musicien` table.

In [6]:

with open(path + 'xJazz_musiciens_all.txt', 'r') as f:
    for line in f:
        line = line.strip()
        line = line.split(';')
        cursor.execute("INSERT INTO Musicien VALUES (?, ?, ?)", (line[0], line[1], line[2]))
        db_jazz.commit()



### Insert organizers into the `Organisateur` table.

In [7]:
with open(path + 'xJazz_organisateurs_all.txt', 'r') as f:
    for line in f:
        line = line.strip()
        line = line.split(';')
        cursor.execute("INSERT INTO Organisateur VALUES (?, ?, ?)", (line[0], line[1].replace('"', ''), line[2]))
        db_jazz.commit()

### Insert venues into the `Salle` table.

In [8]:
with open(path + 'xJazz_salles_all.txt', 'r') as f:
    for line in f:
        line = line.strip()
        line = line.split(';')
        cursor.execute("INSERT INTO Salle VALUES (?, ?, ?)", (line[0], line[1].replace('"', ''), line[2]))
        db_jazz.commit()

In [9]:
with open(path + 'xJazz_morceux_et_arr_all.txt', 'r') as f:
    for line in f:
        line = line.strip()
        line = line.split(';')
        id_morceau = line[0].strip()
        
        nom_musiciens = line[1].replace('\t\t', '\t').split('\t')[1].split('et')
        for nom_musicien in nom_musiciens:
            nom_musicien = nom_musicien.strip()
            # get id musicien
            cursor.execute("SELECT ID_musicien FROM Musicien WHERE Nom_musicien = ?", (nom_musicien,))
            id_musicien = cursor.fetchone()[0]
            if(id_musicien == None):
                print(nom_musicien)

            cursor.execute("INSERT INTO Auteur VALUES (?, ?)", (id_musicien, id_morceau))
            db_jazz.commit()


In [10]:
printTable('Client')
printTable('Morceau')
printTable('Musicien')


--- TABLE Client 

             ID_client             Nom_client             Tel_client 
                     1                 Claire             0769646587 
                     2               Caroline             0912363008 
                     3                 Marcel             0013908883 
                     4                   Eric             0450425837 
                     5                   Sean             0109889300 

--- TABLE Morceau 

            ID_morceau            Nom_morceau 
                     1            Les fraises 
                     2        La joie du Jazz 
                     3                Le truc 
                     4                 Danger 
                     5                  Miaou 

--- TABLE Musicien 

           ID_musicien           Nom_musicien           Tel_musicien 
                     1                  Louis             0615063355 
                     2                Michael             0360717574 
                     3   

In [11]:
printTable('Organisateur')
printTable('Salle')
printTable('Auteur')


--- TABLE Organisateur 

       ID_organisateur       Nom_organisateur       Tel_organisateur 
                     1          Les meilleurs             0698541706 
                     2   Bandits et compagnie             0368638044 
                     3       Le Jazz Ã  faire             0765555168 

--- TABLE Salle 

              ID_salle              Nom_salle          Nombre_Places 
                     1             Le repaire                     30 
                     2          Le vieux soir                     40 
                     3      L'Ã©lÃ©phant rose                    100 
                     4         Le New Orleans                    120 

--- TABLE Auteur 

           ID_musicien             ID_morceau 
                     1                      1 
                     1                      2 
                     3                      3 
                     5                      3 
                     9                      4 


### Read the concerts from concerts.json

In [12]:
import json

with open('donnees_concerts.json') as json_file:
    concerts = json.load(json_file)
    # Iterate through the concert data
    for concert in concerts:
        # Insert into Concert
        cursor.execute("INSERT INTO Concert (ID_concert, Nom_concert, ID_musicien, ID_organisateur) VALUES (?, ?, (SELECT ID_musicien FROM Musicien WHERE Nom_musicien = ?), (SELECT ID_organisateur FROM Organisateur WHERE Nom_organisateur = ?))", 
                       (concert["ID_concert"], concert["Nom_concert"], concert["leader"], concert["organisateur"])) 

        for morceau in concert["Morceaux"]:
            # Insert into Programme
            cursor.execute("INSERT INTO Programme (ID_morceau, ID_concert, ID_arrangeur) VALUES ((SELECT ID_morceau FROM Morceau WHERE Nom_morceau = ?), ?, (SELECT ID_musicien FROM Musicien WHERE Nom_musicien = ?))", 
                           (morceau["Nom"], concert["ID_concert"], morceau["Arrangeur"])) 
            
        for musicien in concert["Musiciens"]:
            # Insert into Participation
            cursor.execute("INSERT INTO Participation (ID_musicien, ID_concert) VALUES ((SELECT ID_musicien FROM Musicien WHERE Nom_musicien = ?), ?)", 
                           (musicien, concert["ID_concert"]))

        # Insert into Seance
        for seance in concert["Seances"]:
            cursor.execute("INSERT INTO Seance (ID_concert, Date, ID_salle) VALUES (?, ?, (SELECT ID_salle FROM Salle WHERE Nom_salle = ?))", 
                           (concert["ID_concert"],seance["date"], seance["salle"])) 

            # Insert into Reservation
            for reservation in seance["Reservations"]:
                try:
                    cursor.execute("INSERT INTO Reservation (ID_concert, Date, ID_client, Num_place, Prix) VALUES (?, ?, (SELECT ID_client FROM Client WHERE Nom_client = ?), ?, ?)", 
                               (concert["ID_concert"],seance["date"], reservation["Nom"], reservation["Place"], reservation["Prix"]))
                except sqlite3.IntegrityError as e:
                    print(e)

In [13]:
printTable('Concert')

printTable('Seance')


--- TABLE Concert 

            ID_concert            Nom_concert            ID_musicien        ID_organisateur 
                     1       La folie du soir                      1                      3 
                     2          Nouvelle Ã¨re                      9                      1 
                     3       Festival d'Ã©tÃ©                      3                      1 

--- TABLE Seance 

            ID_concert                   Date               ID_salle 
                     1             2022/04/15                      1 
                     2             2022/10/05                      3 
                     3             2022/06/21                      4 
                     3             2022/08/07                      2 


In [14]:
printTable('Programme')

printTable('Reservation')


--- TABLE Programme 

            ID_morceau             ID_concert           ID_arrangeur 
                     1                      1                      6 
                     8                      1                      1 
                    12                      1                      1 
                     7                      1                      3 
                    16                      1                      6 

--- TABLE Reservation 

            ID_concert                   Date              ID_client              Num_place                   Prix 
                     1             2022/04/15                     14                      1                     28 
                     1             2022/04/15                      3                      2                     32 
                     1             2022/04/15                      2                      3                     26 
                     1             2022/04/15                      7

In [15]:
printTable('Participation')


--- TABLE Participation 

           ID_musicien             ID_concert 
                     1                      1 
                     3                      1 
                     6                      1 
                     1                      2 
                     3                      2 


In [16]:
def print_requete(sql):
    cursor.execute(sql)
    rows = cursor.fetchall()
    print("\n--- RESULT \n")
    for desc in cursor.description:
        print(desc[0].rjust(22, ' '), end=' ')
    print("")
    for row in rows:
        for value in row:
            print(str(value).rjust(22, ' '), end=' ')
        print("")

### We display the data used to create the concert program

In [17]:
script_1 = '''
select Nom_concert, Nom_musicien, Nom_salle, Nom_organisateur, Date
from
    Concert
    natural join Musicien
    natural join Seance
    natural join Salle
    natural join Organisateur
'''

print_requete(script_1)


--- RESULT 

           Nom_concert           Nom_musicien              Nom_salle       Nom_organisateur                   Date 
      La folie du soir                  Louis             Le repaire       Le Jazz Ã  faire             2022/04/15 
         Nouvelle Ã¨re                   Jaco      L'Ã©lÃ©phant rose          Les meilleurs             2022/10/05 
      Festival d'Ã©tÃ©                  Miles         Le New Orleans          Les meilleurs             2022/06/21 
      Festival d'Ã©tÃ©                  Miles          Le vieux soir          Les meilleurs             2022/08/07 


### Clients who purchased the most tickets
### Method 1

In [18]:
script_2 = '''
select C.ID_client, C.Nom_client, C.Tel_client, count(R.Num_place) as NombreBilletsAchetes
from Client C JOIN Reservation R ON C.ID_client = R.ID_client
group by C.ID_client
order by NombreBilletsAchetes desc
limit 2;
'''

print("les 2 personnes qui ont acheté le plus grand nombre de billets sont: ")
print_requete(script_2)

les 2 personnes qui ont acheté le plus grand nombre de billets sont: 

--- RESULT 

             ID_client             Nom_client             Tel_client   NombreBilletsAchetes 
                    11                 Victor             0756039207                      6 
                    10                  Annie             0815790860                      4 


### Method 2

In [19]:
script_2 = '''
select ID_client, Nom_client, Tel_client, count(Num_place) as NombreBilletsAchetes
from Client NATURAL JOIN Reservation 
group by Client.ID_client
order by NombreBilletsAchetes desc
limit 2;
'''

print("les 2 personnes qui ont acheté le plus grand nombre de billets sont: ")
print_requete(script_2)

les 2 personnes qui ont acheté le plus grand nombre de billets sont: 

--- RESULT 

             ID_client             Nom_client             Tel_client   NombreBilletsAchetes 
                    11                 Victor             0756039207                      6 
                    10                  Annie             0815790860                      4 


---

This query selects the two clients who have purchased the greatest number of tickets across all concerts.

It joins the **Client** and **Réservation** tables to calculate the total number of tickets purchased per client, sorts the results by this count in descending order, and limits the output to the two top buyers.

---

### Method 3

In [20]:
script_2='''
with ClassementClients as (select ID_client, Nom_client, Tel_client, count(Num_place) as NombreBilletsAchetes,
    rank() over (order by count(Num_place) desc) as Classement
    from Client natural join Reservation 
    group by ID_client, Nom_client, Tel_client)
    
select ID_client, Nom_client, Tel_client, NombreBilletsAchetes
from ClassementClients where Classement < 3;
'''

print("les 2 personnes qui ont acheté le plus grand nombre de billets sont: ")
print_requete(script_2)

les 2 personnes qui ont acheté le plus grand nombre de billets sont: 

--- RESULT 

             ID_client             Nom_client             Tel_client   NombreBilletsAchetes 
                    11                 Victor             0756039207                      6 
                    10                  Annie             0815790860                      4 


### Clients who purchased tickets for all sessions across all concerts

### Method 1

In [21]:
script_3 = '''
select Client.ID_client, Client.Nom_client, Client.Tel_client, 
    count (distinct Seance.Date) as Nombre_Seances_Réservées
    
from Client inner join Reservation on Client.ID_client = Reservation.ID_client
            inner join Seance ON Reservation.ID_concert = Seance.ID_concert AND Reservation.Date = Seance.Date
group by Client.ID_client

having count(distinct Seance.Date) = (select count(*) from Seance)
and not exists ( select 1 from Concert c
    where ( select count(*) from Seance where ID_concert = c.ID_concert) > (select count(distinct s.Date) 
                                                                            from Reservation r
                                                                            inner join Seance s on r.ID_concert = s.ID_concert 
                                                                            and r.Date = s.Date
                                                                            where r.ID_client = Client.ID_client 
                                                                            and s.ID_concert = c.ID_concert)
);


'''

print("Les clients qui ont acheté des billets à toutes les séances des tous les concerts sont: ")
print_requete(script_3)

Les clients qui ont acheté des billets à toutes les séances des tous les concerts sont: 

--- RESULT 

             ID_client             Nom_client             Tel_client Nombre_Seances_Réservées 
                    10                  Annie             0815790860                      4 


### Method 2

In [22]:

script_3 = '''
with Client_Seance as (select c.ID_client, s.ID_concert, s.Date
    from Client c cross join Seance s)
    ,

Reservations_Count as (select cs.ID_client, count (r.Date) as Nombre_Seances_Réservées
    from Client_Seance cs left join Reservation r on cs.ID_client = r.ID_client 
                                                  and cs.ID_concert = r.ID_concert 
                                                  and cs.Date = r.Date
    group by cs.ID_client
    having count(r.Date) = (select count(*) from Seance))

select cl.ID_client, cl.Nom_client, cl.Tel_client, rc.Nombre_Seances_Réservées
from Client cl join Reservations_Count rc on cl.ID_client = rc.ID_client
'''

print("Les clients qui ont acheté des billets à toutes les séances des tous les concerts sont: ")
print_requete(script_3)

Les clients qui ont acheté des billets à toutes les séances des tous les concerts sont: 

--- RESULT 

             ID_client             Nom_client             Tel_client Nombre_Seances_Réservées 
                    10                  Annie             0815790860                      4 


---

Here, we used Common Table Expressions (CTEs) to identify clients who purchased tickets for every session of every concert.

The first CTE, `Client_Session`, generates a list of all possible combinations of clients and sessions, representing the complete set of sessions each client could potentially attend.

The second CTE, `Reservations_Count`, calculates the number of sessions reserved by each client and filters for those who reserved a number of sessions equal to the total available sessions. Finally, the main query joins the `Reservations_Count` CTE with the `Client` table to retrieve the results.

---

### Number of concert sessions organized by each organizer
### Method 1

In [23]:
script_4 = '''
select O.ID_Organisateur, O.Nom_Organisateur, count( S.ID_concert) AS NombreSeances
from Organisateur O join Concert C ON O.ID_Organisateur = C.ID_Organisateur join Seance S ON C.ID_Concert = S.ID_Concert
group by O.ID_Organisateur, O.Nom_Organisateur;
'''

print("Le nombre de séances de concerts organisés par chaque organisateur sont: ")
print_requete(script_4)

Le nombre de séances de concerts organisés par chaque organisateur sont: 

--- RESULT 

       ID_organisateur       Nom_organisateur          NombreSeances 
                     1          Les meilleurs                      3 
                     3       Le Jazz Ã  faire                      1 


### Method 2


In [24]:
script_4 = ''' select ID_Organisateur, Nom_Organisateur, count ( ID_concert) as NombreSeances
from Organisateur natural join Concert  
                    natural join (select ID_Concert from Seance)
group by ID_Organisateur, Nom_Organisateur;
'''

print("Le nombre de séances de concerts organisés par chaque organisateur sont: ")
print_requete(script_4)

Le nombre de séances de concerts organisés par chaque organisateur sont: 

--- RESULT 

       ID_organisateur       Nom_organisateur          NombreSeances 
                     1          Les meilleurs                      3 
                     3       Le Jazz Ã  faire                      1 


---

The query uses natural joins to calculate the number of sessions associated with each organizer. It begins by linking the `Organisateur` and `Concert` tables to retrieve all concerts organized by each organizer, then incorporates data from the `Séance` table to include the number of sessions for each concert.

Using the `COUNT` aggregation, it tallies the sessions per organizer and groups the results by the organizer's ID and name.

---

### Musicians who have never arranged a piece
### Method 1

In [25]:
script_5 = '''
select m.ID_musicien, m.Nom_musicien from Musicien m
where not exists (select 1 from Programme p where p.ID_arrangeur = m.ID_musicien);
'''

print("Les musicien n'ont jamais arrangé un morceau sont: ")
print_requete(script_5)

Les musicien n'ont jamais arrangé un morceau sont: 

--- RESULT 

           ID_musicien           Nom_musicien 
                     2                Michael 
                     4                Ernesto 
                     5                  Keith 
                     8                  Fruko 
                    10                Douglas 
                    12              Stanislas 


### Method 2


In [26]:
script_5 = '''
select m.ID_musicien, m.Nom_musicien 
from Musicien m left join Programme p on m.ID_musicien = p.ID_arrangeur
where p.ID_morceau is null;
'''

print("Les musicien n'ont jamais arrangé un morceau sont: ")
print_requete(script_5)

Les musicien n'ont jamais arrangé un morceau sont: 

--- RESULT 

           ID_musicien           Nom_musicien 
                     2                Michael 
                     4                Ernesto 
                     5                  Keith 
                     8                  Fruko 
                    10                Douglas 
                    12              Stanislas 


---

This query uses a left join to identify musicians who have never arranged a piece. It links the `Musicien` table to the `Programme` table on the musician's ID and checks for the absence of a match in the `Programme` table (indicated by `p.ID_morceau IS NULL`).

This selects only the musicians who have no pieces associated with their name as an arranger, indicating they have never arranged any pieces.

---

### How much the revenue ticket sales generated for each concert
### Method 1

In [27]:

script_6 = '''
select co.ID_concert, co.Nom_concert, SUM(r.Prix) as Revenu_Total
from Concert co join Reservation r on co.ID_concert = r.ID_concert
group by co.ID_concert, co.Nom_concert;
'''

print("Les ventes de places pour chaque concert ont rapporté le suivant: ")
print_requete(script_6)

Les ventes de places pour chaque concert ont rapporté le suivant: 

--- RESULT 

            ID_concert            Nom_concert           Revenu_Total 
                     1       La folie du soir                    342 
                     2          Nouvelle Ã¨re                    315 
                     3       Festival d'Ã©tÃ©                    311 


### Method 2

In [28]:
script_6 = '''
select ID_concert, Nom_concert, sum(Prix) as Revenu_Total
from Concert natural join Reservation
group by ID_concert, Nom_concert;
'''

print("Les ventes de places pour chaque concert ont rapporté le suivant: ")
print_requete(script_6)

Les ventes de places pour chaque concert ont rapporté le suivant: 

--- RESULT 

            ID_concert            Nom_concert           Revenu_Total 
                     1       La folie du soir                    342 
                     2          Nouvelle Ã¨re                    315 
                     3       Festival d'Ã©tÃ©                    311 


---

In this query, a natural join is used to calculate the total revenue for each concert by summing up the price of tickets sold. This join automatically links the `Concert` and `Réservation` tables based on their common columns, here the concert ID.

Subsequently, the `SUM` function is applied to total all ticket prices (`Prix`) for each concert, grouping the results by the concert ID and name. Thus, the final output shows the total revenue for every concert listed in the database.

---

### 7. Pieces that are played at all concerts

### Method 1

In [29]:
script_7 = '''
select m.ID_morceau, m.Nom_morceau from Morceau m
where not exists (select 1 from Concert c where not exists (select 1
                                                            from Programme p
                                                            where p.ID_morceau = m.ID_morceau
                                                            and p.ID_concert = c.ID_concert)
);
'''
print("Les morceaux joués à tous les concerts sont: ")
print_requete(script_7)

Les morceaux joués à tous les concerts sont: 

--- RESULT 

            ID_morceau            Nom_morceau 
                     7                  Ouais 
                    12                   Boum 
                    14                 La vie 
                    16               Kokoriko 


### Method 2

In [30]:
script_7 = '''
select m.ID_morceau, m.Nom_morceau
from Morceau m join Programme p on m.ID_morceau = p.ID_morceau
group by m.ID_morceau, m.Nom_morceau
having count(distinct p.ID_concert) = (select count (distinct ID_concert) from Concert);
'''
print("Les morceaux joués à tous les concerts sont: ")
print_requete(script_7)



Les morceaux joués à tous les concerts sont: 

--- RESULT 

            ID_morceau            Nom_morceau 
                     7                  Ouais 
                    12                   Boum 
                    14                 La vie 
                    16               Kokoriko 


---

This query identifies the pieces that have been played at all concerts recorded in the database. It performs a join between the `Morceau` table and the `Programme` table using the piece identifier (`ID_morceau`). This links each piece to all the concerts where it has been performed.

Furthermore, the query groups the results by `ID_morceau` and piece name (attributes from the `Morceau` table), which is necessary to perform calculations for each piece individually.

Additionally, the `COUNT(DISTINCT p.ID_concert)` function is used to count the number of distinct concerts where each piece has been played. The `HAVING` clause is then applied to filter the pieces by comparing the number of concerts where a piece was performed with the total number of distinct concerts (`SELECT COUNT(DISTINCT ID_concert) FROM Concert`). Only the pieces performed in a number of concerts equal to the total count of distinct concerts are selected.

---

In [31]:

script_8 = '''
select Nom_musicien, Tel_musicien from musicien natural join reservation natural join client
where Tel_musicien = Tel_client
'''

print("Les musiciens parmi les clients sont: ")
print_requete(script_8)

Les musiciens parmi les clients sont: 

--- RESULT 

          Nom_musicien           Tel_musicien 
             Stanislas             0622181420 
                 Omara             0316508123 


---

This query is designed to identify musicians who are also clients, based on the premise that if a musician and a client share the same phone number, they are in fact the same person.

To achieve this, we use a natural join (`NATURAL JOIN`) to combine the `Musicien`, `Réservation`, and `Client` tables. This join automatically matches columns with the same names across these tables.

The `WHERE Tel_musicien = Tel_client` clause filters the results to retain only cases where the musician's phone number matches the client's, thereby suggesting they are the same individual.

Finally, we select the musician's name (`Nom_musicien`) and phone number (`Tel_musicien`) for those musicians who meet this condition.

---

### Revenue per concert session

### Method 1

In [32]:
script_9 = '''
select s.ID_concert, s.Date, sum(r.Prix) as Chiffre_Affaires, count(*) as Nombre_billets_Vendus
from Seance s join Reservation r on s.ID_concert = r.ID_concert and s.Date = r.Date
group by s.ID_concert, s.Date;
'''

print("Le chiffre d’affaires par séance de concert est représenté ci-dessous: ")
print_requete(script_9)

Le chiffre d’affaires par séance de concert est représenté ci-dessous: 

--- RESULT 

            ID_concert                   Date       Chiffre_Affaires  Nombre_billets_Vendus 
                     1             2022/04/15                    342                     14 
                     2             2022/10/05                    315                     16 
                     3             2022/06/21                    163                     10 
                     3             2022/08/07                    148                     10 


### Method 2

In [33]:
###2eme methode
script_9 = '''
select s.ID_concert, s.Date, (select sum(r.Prix) from Reservation r
where r.ID_concert = s.ID_concert and r.Date = s.Date) as Chiffre_Affaires
from Seance s;
'''

print("Le chiffre d’affaires par séance de concert est représenté ci-dessous: ")
print_requete(script_9)

Le chiffre d’affaires par séance de concert est représenté ci-dessous: 

--- RESULT 

            ID_concert                   Date       Chiffre_Affaires 
                     1             2022/04/15                    342 
                     2             2022/10/05                    315 
                     3             2022/06/21                    163 
                     3             2022/08/07                    148 


### Method 3


In [34]:
script_9 = '''
select distinct s.ID_concert, s.Date,
    count(r.ID_client) over (partition by s.ID_concert, s.Date) as Nombre_Billets_Vendus,
    sum(r.Prix) over (partition by s.ID_concert, s.Date) as Chiffre_Affaires
from Seance s join Reservation r ON s.ID_concert = r.ID_concert AND s.Date = r.Date;
'''

print("Le chiffre d’affaires par séance de concert est représenté ci-dessous: ")
print_requete(script_9)

Le chiffre d’affaires par séance de concert est représenté ci-dessous: 

--- RESULT 

            ID_concert                   Date  Nombre_Billets_Vendus       Chiffre_Affaires 
                     1             2022/04/15                     14                    342 
                     2             2022/10/05                     16                    315 
                     3             2022/06/21                     10                    163 
                     3             2022/08/07                     10                    148 


---

This query is designed to calculate the revenue for each concert session. To achieve this, we perform a join between the `Séance` and `Réservation` tables using both the concert ID (`ID_concert`) and the session date (`Date`). This links each reservation to its specific concert session.

We also selected the concert identifier and the session date to uniquely identify each session. Additionally, we calculate the total revenue generated from ticket sales for each session. The sum of reservation prices (`r.Prix`) provides the total revenue.

We included `COUNT(*) AS Nombre_billets_Vendus` to count the total number of tickets sold for each session. This `COUNT(*)` function counts all rows (reservations) belonging to each session group.

Finally, our results are grouped by the concert identifier and the session date, allowing us to obtain distinct results for each concert session.

---

### Organizer Who Earned the Most (Based on 20% of Concert Revenue)


In [35]:
script_10 = '''
select o.ID_organisateur, o.Nom_organisateur, sum(r.Prix) * 0.2 
as Revenu_Organisateur
from Organisateur o join Concert co on o.ID_organisateur = co.ID_organisateur
join Reservation r on co.ID_concert = r.ID_concert
group by o.ID_organisateur, o.Nom_organisateur
order by Revenu_Organisateur desc
limit 1;
'''
print("L'organisateur ayant gagné le plus sachant qu'il lui revient en moyenne 20% du chiffre d’affaires du concert est: ")
print_requete(script_10)

L'organisateur ayant gagné le plus sachant qu'il lui revient en moyenne 20% du chiffre d’affaires du concert est: 

--- RESULT 

       ID_organisateur       Nom_organisateur    Revenu_Organisateur 
                     1          Les meilleurs                  125.2 


---

This query is designed to identify the concert organizer who generated the most revenue, given that they receive on average 20% of the revenue from each concert.

To achieve this, we first join the `Organisateur` table to the `Concert` table on the organizer's ID, then link the `Concert` table to the `Réservation` table on the concert ID. This join operation connects each organizer to the concerts they organized and the corresponding reservations.

We calculate the organizer's revenue for each group (organizer) by summing all ticket prices sold for the concerts they organized (`SUM(r.Prix)`) and taking 20% of that total (multiplied by 0.2), since the organizer receives 20% of the revenue.

The results are then grouped by the organizer's identifier and name to determine the total revenue per organizer. Finally, we sort the organizers by their revenue in descending order and limit the results to the first record, which corresponds to the organizer who earned the most money.

---